In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
! gsutil ls gs://14oct/data/

gs://14oct/data/
gs://14oct/data/ml-100k.zip


In [4]:
! gsutil cp gs://14oct/data/ml-100k.zip ./

Copying gs://14oct/data/ml-100k.zip...
- [1 files][  4.7 MiB/  4.7 MiB]                                                
Operation completed over 1 objects/4.7 MiB.                                      


In [5]:
! unzip *.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [7]:
! head -3 ml-100k/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067


In [19]:
! wc -l ml-100k/u.user

943 ml-100k/u.user


In [16]:
! hdfs dfs -put ml-100k/u.user /

In [22]:
spark.read.option('delimiter','|').csv('/u.user').count()

943

In [21]:
spark.read.option('delimiter','|').csv('/u.user').show()

+---+---+---+-------------+-----+
|_c0|_c1|_c2|          _c3|  _c4|
+---+---+---+-------------+-----+
|  1| 24|  M|   technician|85711|
|  2| 53|  F|        other|94043|
|  3| 23|  M|       writer|32067|
|  4| 24|  M|   technician|43537|
|  5| 33|  F|        other|15213|
|  6| 42|  M|    executive|98101|
|  7| 57|  M|administrator|91344|
|  8| 36|  M|administrator|05201|
|  9| 29|  M|      student|01002|
| 10| 53|  M|       lawyer|90703|
| 11| 39|  F|        other|30329|
| 12| 28|  F|        other|06405|
| 13| 47|  M|     educator|29206|
| 14| 45|  M|    scientist|55106|
| 15| 49|  F|     educator|97301|
| 16| 21|  M|entertainment|10309|
| 17| 30|  M|   programmer|06355|
| 18| 35|  F|        other|37212|
| 19| 40|  M|    librarian|02138|
| 20| 42|  F|    homemaker|95660|
+---+---+---+-------------+-----+
only showing top 20 rows



In [33]:
customer_raw_df = spark.read.option('delimiter','|').option('inferSchema','true').csv('/u.user')\
.withColumnRenamed('_c0','id')\
.withColumnRenamed('_c1','age')\
.withColumnRenamed('_c2','gender')\
.withColumnRenamed('_c3','job')\
.withColumnRenamed('_c4','zip_code')

In [34]:
customer_raw_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- job: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [38]:
customer_raw_df.show()

+---+---+------+-------------+--------+
| id|age|gender|          job|zip_code|
+---+---+------+-------------+--------+
|  1| 24|     M|   technician|   85711|
|  2| 53|     F|        other|   94043|
|  3| 23|     M|       writer|   32067|
|  4| 24|     M|   technician|   43537|
|  5| 33|     F|        other|   15213|
|  6| 42|     M|    executive|   98101|
|  7| 57|     M|administrator|   91344|
|  8| 36|     M|administrator|   05201|
|  9| 29|     M|      student|   01002|
| 10| 53|     M|       lawyer|   90703|
| 11| 39|     F|        other|   30329|
| 12| 28|     F|        other|   06405|
| 13| 47|     M|     educator|   29206|
| 14| 45|     M|    scientist|   55106|
| 15| 49|     F|     educator|   97301|
| 16| 21|     M|entertainment|   10309|
| 17| 30|     M|   programmer|   06355|
| 18| 35|     F|        other|   37212|
| 19| 40|     M|    librarian|   02138|
| 20| 42|     F|    homemaker|   95660|
+---+---+------+-------------+--------+
only showing top 20 rows



In [35]:
customer_raw_df.count()

943

In [37]:
customer_raw_df.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  273|
|     M|  670|
+------+-----+



In [39]:
from pyspark.sql import functions as sparkf

In [43]:
customer_raw_df.groupBy('gender').agg(sparkf.count('gender'),sparkf.avg('age')).show()

+------+-------------+------------------+
|gender|count(gender)|          avg(age)|
+------+-------------+------------------+
|     F|          273| 33.81318681318681|
|     M|          670|34.149253731343286|
+------+-------------+------------------+

